# Data Processing

In this notebook, we will prepare the dataset for modeling.  

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

**A. Data daily**

**1. Overview of Data after Data Understanding**

In [81]:
# Load the dataset
df_daily = pd.read_excel(r'../data/processed/data_daily_after_basic_understand.xlsx')

In [82]:
# Xác định loại dữ liệu
numerical_features = df_daily.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = df_daily.select_dtypes(include=['object']).columns.tolist()

print("Numeric features:", numerical_features)
print("Categorical features:", categorical_features)

Numeric features: ['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipcover', 'windgust', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'moonphase']
Categorical features: ['conditions', 'description', 'icon', 'stations']


In [83]:
df_daily['datetime'] = pd.to_datetime(df_daily['datetime'])
df_daily = df_daily.sort_values('datetime').reset_index(drop=True)

**2. Drop missing columns**


In [84]:
# Kiểm tra missing values và duplicates
print("Số lượng giá trị thiếu:\n", df_daily.isnull().sum())
print("\nTỷ lệ thiếu (%):\n", (df_daily.isnull().mean() * 100).round(2))

dupes_daily = df_daily.duplicated().sum()
print(f"\nSố dòng trùng lặp: {dupes_daily}")

Số lượng giá trị thiếu:
 datetime               0
tempmax                0
tempmin                0
temp                   0
feelslikemax           0
feelslikemin           0
feelslike              0
dew                    0
humidity               0
precip                 0
precipcover            0
windgust               0
windspeed              0
winddir                0
sealevelpressure       0
cloudcover             0
visibility             0
solarradiation         0
solarenergy            0
uvindex                0
severerisk          2566
sunrise                0
sunset                 0
moonphase              0
conditions             0
description            0
icon                   0
stations               0
dtype: int64

Tỷ lệ thiếu (%):
 datetime             0.00
tempmax              0.00
tempmin              0.00
temp                 0.00
feelslikemax         0.00
feelslikemin         0.00
feelslike            0.00
dew                  0.00
humidity             0.00
precip   

During the data exploration process, I observed that **only the `severerisk` column contains missing values**, with a **missing rate as high as 65.34%**. Such a high proportion of missing data suggests that the column provides limited informational value and may negatively impact model performance if retained. Furthermore, correlation analysis shows that `severerisk` has little to no relationship with the target variable. Keeping it would add noise rather than value to the model.  

Given the high missing rate, weak correlation, and lack of statistical reliability,  
the `severerisk` column will be **dropped** from the dataset prior to feature engineering and model training.

In [85]:
# Loại bỏ cột severisk
df_daily = df_daily.drop(columns=['severerisk'])
# Kiểm tra lại kết quả
print("Kích thước dữ liệu sau khi drop cột 'severerisk':", df_daily.shape)

Kích thước dữ liệu sau khi drop cột 'severerisk': (3927, 27)


**3. Train-test split**

In [86]:
# Chia train-test split theo thời gian, trong đó khoảng cách tập train và test là 7 ngày để tránh data leakage do tính chất thời gian của dữ liệu, temp là biến mục tiêu
train_size = int(len(df_daily) * 0.8)
# Tập huấn luyện kết thúc trước 7 ngày so với tập kiểm tra
train_data = df_daily.iloc[:train_size - 7] 
# Tập kiểm tra bắt đầu từ ngày thứ 7 sau tập huấn luyện đến hết
test_data = df_daily.iloc[train_size:]
print(f"Kích thước tập huấn luyện: {train_data.shape}")
print(f"Kích thước tập kiểm tra: {test_data.shape}")
print(f"Khoảng thời gian tập huấn luyện: {train_data['datetime'].min()} đến {train_data['datetime'].max()}")
print(f"Khoảng thời gian tập kiểm tra: {test_data['datetime'].min()} đến {test_data['datetime'].max()}")

# temp là biến mục tiêu
feature_cols = df_daily.columns.drop(['temp', 'datetime'])  # Loại bỏ cột temp khỏi features
X_train = train_data[feature_cols]
y_train = train_data['temp']
X_test = test_data[feature_cols]
y_test = test_data['temp']

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

Kích thước tập huấn luyện: (3134, 27)
Kích thước tập kiểm tra: (786, 27)
Khoảng thời gian tập huấn luyện: 2015-01-01 00:00:00 đến 2023-07-31 00:00:00
Khoảng thời gian tập kiểm tra: 2023-08-08 00:00:00 đến 2025-10-01 00:00:00
X_train shape: (3134, 25), y_train shape: (3134,)
X_test shape: (786, 25), y_test shape: (786,)


**4. Observing and handling outliers**

In [87]:
# # Phát hiện outliers bằng phương pháp IQR
numeric_cols_d = df_daily.select_dtypes(include=[np.number]).columns

Q1_d = df_daily[numeric_cols_d].quantile(0.25)
Q3_d = df_daily[numeric_cols_d].quantile(0.75)
IQR_d = Q3_d - Q1_d

outliers_d = ((df_daily[numeric_cols_d] < (Q1_d - 1.5 * IQR_d)) | 
              (df_daily[numeric_cols_d] > (Q3_d + 1.5 * IQR_d))).sum()

print("Số lượng outliers (theo từng cột):\n", outliers_d)

Số lượng outliers (theo từng cột):
 tempmax               3
tempmin               1
temp                  2
feelslikemax          0
feelslikemin         20
feelslike             0
dew                  41
humidity             91
precip              639
precipcover         169
windgust            117
windspeed            59
winddir             664
sealevelpressure      1
cloudcover            0
visibility          119
solarradiation        0
solarenergy           0
uvindex               0
moonphase             0
dtype: int64


In [88]:
# Phân loại các cột dựa trên số lượng outliers
def classify_columns_by_outliers(X, outlier_threshold=0.05):
    """
    Phân loại cột numeric thành high outliers và low outliers
    outlier_threshold: tỷ lệ outliers để phân loại (mặc định 5%)
    """
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    Q1 = X[numeric_cols].quantile(0.25)
    Q3 = X[numeric_cols].quantile(0.75)
    IQR = Q3 - Q1
    
    outlier_counts = ((X[numeric_cols] < (Q1 - 1.5 * IQR)) | 
                      (X[numeric_cols] > (Q3 + 1.5 * IQR))).sum()
    outlier_ratios = outlier_counts / len(X)
    
    high_outlier_cols = outlier_ratios[outlier_ratios > outlier_threshold].index.tolist()
    low_outlier_cols = outlier_ratios[outlier_ratios <= outlier_threshold].index.tolist()
    
    return high_outlier_cols, low_outlier_cols, outlier_ratios

# Phân loại cột cho dữ liệu daily
high_outlier_cols, low_outlier_cols, outlier_ratios = classify_columns_by_outliers(X_train, outlier_threshold=0.05)

print("=== Phân loại cột dựa trên outliers ===")
print(f"\nCột có nhiều outliers (>5%): {high_outlier_cols}")
print(f"\nCột có ít outliers (<=5%): {low_outlier_cols}")
print(f"\nTỷ lệ outliers chi tiết:\n{outlier_ratios.sort_values(ascending=False)}")

# Xác định cột categorical
nominal_cols = X_train.select_dtypes(include=['object']).columns.tolist()
ordinal_cols = []  # Nếu có cột ordinal, khai báo ở đây

print(f"\nCột nominal: {nominal_cols}")
print(f"Cột ordinal: {ordinal_cols}")

=== Phân loại cột dựa trên outliers ===

Cột có nhiều outliers (>5%): ['precip', 'winddir']

Cột có ít outliers (<=5%): ['tempmax', 'tempmin', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity', 'precipcover', 'windgust', 'windspeed', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'moonphase']

Tỷ lệ outliers chi tiết:
winddir             0.166879
precip              0.166879
precipcover         0.037652
windgust            0.029036
visibility          0.027760
humidity            0.020740
windspeed           0.012763
cloudcover          0.010530
dew                 0.009572
feelslikemin        0.006063
sealevelpressure    0.000638
tempmax             0.000638
tempmin             0.000319
feelslike           0.000000
feelslikemax        0.000000
solarradiation      0.000000
solarenergy         0.000000
uvindex             0.000000
moonphase           0.000000
dtype: float64

Cột nominal: ['conditions', 'description', 'icon', 'sta

In [89]:
from sklearn.preprocessing import RobustScaler, StandardScaler, OneHotEncoder, OrdinalEncoder

# Tách các cột datetime
datetime_cols = X_train.select_dtypes(include=['datetime']).columns.tolist()

# Tách các cột không phải datetime (numeric và categorical)
train_non_datetime = train_data.drop(columns=datetime_cols)
test_non_datetime = test_data.drop(columns=datetime_cols)

# Tạo ColumnTransformer
transformers = []

# 1. RobustScaler cho numeric có nhiều outliers
if high_outlier_cols:
    transformers.append(('robust', RobustScaler(), high_outlier_cols))

# 2. StandardScaler cho numeric có ít outliers
if low_outlier_cols:
    transformers.append(('standard', StandardScaler(), low_outlier_cols))

# 3. OneHotEncoder cho nominal
if nominal_cols:
    transformers.append(('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), nominal_cols))

# 4. OrdinalEncoder cho ordinal (nếu có)
if ordinal_cols:
    transformers.append(('ordinal', OrdinalEncoder(), ordinal_cols))

# Tạo ColumnTransformer (passthrough để giữ nguyên các cột không được transform)
preprocessor = ColumnTransformer(
    transformers=transformers,
    remainder='passthrough', verbose_feature_names_out=False  # Giữ nguyên các cột không được transform
)

# ===== FIT VÀ TRANSFORM =====
print("\n=== Bắt đầu fit và transform dữ liệu ===")
train_transformed = preprocessor.fit_transform(train_non_datetime)

# Transform tập TEST (dùng parameters đã học từ train)
test_transformed = preprocessor.transform(test_non_datetime)

print(f"\nKích thước sau khi transform:")
print(f"  X_train: {train_non_datetime.shape} -> {train_transformed.shape}")
print(f"  X_test: {test_non_datetime.shape} -> {test_transformed.shape}")

# Kết hợp lại các cột datetime vào kết quả cuối cùng
# Đảm bảo rằng các cột datetime sẽ không bị thay đổi
train_transformed = pd.DataFrame(train_transformed)
test_transformed = pd.DataFrame(test_transformed)

# Thêm lại cột datetime vào kết quả cuối cùng
train_transformed[datetime_cols] = train_data[datetime_cols].reset_index(drop=True)
test_transformed[datetime_cols] = test_data[datetime_cols].reset_index(drop=True)

# Hiển thị kết quả
print(f"\nKích thước sau khi thêm cột datetime vào:")
print(f"  X_train: {train_transformed.shape}")
print(f"  X_test: {test_transformed.shape}")


=== Bắt đầu fit và transform dữ liệu ===

Kích thước sau khi transform:
  X_train: (3134, 25) -> (3134, 78)
  X_test: (786, 25) -> (786, 78)

Kích thước sau khi thêm cột datetime vào:
  X_train: (3134, 80)
  X_test: (786, 80)


In [90]:
# ...existing code...
# ===== Sửa lỗi: fit/transform + giữ tên cột + giữ datetime =====
import os
from pandas.api import types as pd_types

# Fit trên dữ liệu không chứa datetime (nếu chưa fit)
preprocessor.fit(train_non_datetime)

train_arr = preprocessor.transform(train_non_datetime)
test_arr = preprocessor.transform(test_non_datetime)

# Tạo tên feature an toàn
try:
    # sklearn >=1.0
    feature_names = list(preprocessor.get_feature_names_out(train_non_datetime.columns))
except Exception:
    feature_names = []
    used_cols = []
    for name, trans, cols in preprocessor.transformers_:
        if name == 'remainder':
            continue
        cols_list = list(cols) if isinstance(cols, (list, tuple, np.ndarray)) else [cols]
        used_cols.extend(cols_list)
        fitted = preprocessor.named_transformers_.get(name, None)
        if fitted is not None and hasattr(fitted, "get_feature_names_out"):
            try:
                out = fitted.get_feature_names_out(cols_list)
                feature_names.extend(list(out))
            except Exception:
                feature_names.extend([f"{name}__{c}" for c in cols_list])
        else:
            feature_names.extend([f"{name}__{c}" for c in cols_list])
    # thêm passthrough nếu có
    if getattr(preprocessor, "remainder", None) == 'passthrough':
        passthrough = [c for c in train_non_datetime.columns if c not in used_cols]
        feature_names.extend(passthrough)

# Fallback nếu mismatch kích thước
if len(feature_names) != train_arr.shape[1]:
    feature_names = [f"feat_{i}" for i in range(train_arr.shape[1])]

# Chuyển về DataFrame và giữ index gốc
train_transformed = pd.DataFrame(train_arr, columns=feature_names, index=train_non_datetime.index)
test_transformed = pd.DataFrame(test_arr, columns=feature_names, index=test_non_datetime.index)

# Lấy cột datetime từ train_data/test_data (giữ dtype)
datetime_cols = [c for c in train_data.columns if pd_types.is_datetime64_any_dtype(train_data[c])]

for dt in datetime_cols:
    # nếu tên datetime vô tình có trong feature_names thì ghi đè bằng giá trị gốc
    train_transformed[dt] = train_data.loc[train_transformed.index, dt].values
    test_transformed[dt] = test_data.loc[test_transformed.index, dt].values

# Đảm bảo thứ tự cột: feature_names (không chứa datetime) + datetime_cols
final_feature_names = [c for c in feature_names if c not in datetime_cols] + datetime_cols
train_transformed = train_transformed[final_feature_names]
test_transformed = test_transformed[final_feature_names]

# Thêm target 'temp' lại vào nếu cần và lưu
train_data_transformed = train_transformed.copy()
train_data_transformed['temp'] = train_data.loc[train_transformed.index, 'temp'].values
test_data_transformed = test_transformed.copy()
test_data_transformed['temp'] = test_data.loc[test_transformed.index, 'temp'].values

save_dir = '../data/processed/'
os.makedirs(save_dir, exist_ok=True)
train_data_transformed.to_excel(save_dir + 'train_data.xlsx', index=False)
test_data_transformed.to_excel(save_dir + 'test_data.xlsx', index=False)
X_train_transformed = train_transformed.copy()
X_test_transformed = test_transformed.copy()
X_train_transformed.to_excel(save_dir + 'X_train.xlsx', index=False)
X_test_transformed.to_excel(save_dir + 'X_test.xlsx', index=False)
# ...existing code...

In [91]:
# Save the processed data (train_data_transformed / test_data_transformed already created above)
import os
save_dir = '../data/processed/'
os.makedirs(save_dir, exist_ok=True)

# Save raw copies if needed
train_data.to_excel(save_dir + 'train_data_raw.xlsx', index=False)
test_data.to_excel(save_dir + 'test_data_raw.xlsx', index=False)

# Helper to convert transformed array -> DataFrame with sensible column names
def to_df(arr, index, fallback_prefix='feature'):
    if isinstance(arr, np.ndarray):
        if 'feature_names_out' in globals() and len(feature_names_out) == arr.shape[1]:
            cols = feature_names_out
        else:
            cols = [f'{fallback_prefix}_{i}' for i in range(arr.shape[1])]
        return pd.DataFrame(arr, columns=cols, index=index)
    elif isinstance(arr, pd.DataFrame):
        return arr.copy()
    else:
        raise TypeError("Transformed data must be numpy.ndarray or pandas.DataFrame")

# Convert and save train_data_transformed / test_data_transformed (no X/y combine)
train_data.to_excel(save_dir + 'train_data.xlsx', index=False)
test_data.to_excel(save_dir + 'test_data.xlsx', index=False)

train_index = X_train.index if isinstance(X_train, pd.DataFrame) else range(X_train.shape[0])
test_index  = X_test.index  if isinstance(X_test, pd.DataFrame)  else range(X_test.shape[0])

# Chuyển X_train_transformed / X_test_transformed -> DataFrame
if 'X_train_transformed' in globals():
    train_data_transformed_df = to_df(X_train_transformed, index=train_index, fallback_prefix='Xtrain_feat')
    train_data_transformed_df.to_excel(save_dir + 'train_data_transformed.xlsx', index=False)

if 'X_test_transformed' in globals():
    test_data_transformed_df = to_df(X_test_transformed, index=test_index, fallback_prefix='Xtest_feat')
    test_data_transformed_df.to_excel(save_dir + 'test_data_transformed.xlsx', index=False)

train_data_transformed_df.to_excel(save_dir + 'train_data.xlsx', index=False)
test_data_transformed_df.to_excel(save_dir + 'test_data.xlsx', index=False)

# Chuyển y_train / y_test thành Series nếu đang là ndarray
y_train_series = pd.Series(y_train) if isinstance(y_train, np.ndarray) else y_train.squeeze()
y_test_series  = pd.Series(y_test)  if isinstance(y_test, np.ndarray)  else y_test.squeeze()

# Đặt tên cột nếu chưa có
if y_train_series.name is None:
    y_train_series.name = 'target'
if y_test_series.name is None:
    y_test_series.name = 'target'

# Lưu ra Excel
y_train_series.to_frame().to_excel(save_dir + 'y_train.xlsx', index=False)
y_test_series.to_frame().to_excel(save_dir + 'y_test.xlsx', index=False)


In [92]:
feature_cols = train_transformed.columns.drop(['temp', 'datetime']) 
X_train_transformed = train_transformed[feature_cols]
y_train_transformed = train_transformed['temp']
X_test_transformed = test_transformed[feature_cols]
y_test_transformed = test_transformed['temp']

**TRAIN MODEL**

In [94]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

target = train_data_transformed['temp'].values

n_past = 7
n_future = 5

X, y = [], []
for i in range(len(target) - n_past - n_future + 1):
    X.append(target[i:i+n_past])
    y.append(target[i+n_past:i+n_past+n_future])

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)

split_idx = int(len(X) * 0.8)
X_train_seq, X_test_seq = X[:split_idx], X[split_idx:]
y_train_seq, y_test_seq = y[:split_idx], y[split_idx:]

base_model = RandomForestRegressor(n_estimators=200, random_state=42)
multi_model = MultiOutputRegressor(base_model)

multi_model.fit(X_train_seq, y_train_seq)

y_pred_seq = multi_model.predict(X_test_seq)

mse = mean_squared_error(y_test_seq, y_pred_seq, multioutput='raw_values')
mae = mean_absolute_error(y_test_seq, y_pred_seq, multioutput='raw_values')

for i in range(n_future):
    print(f"Ngày {i+1}: MSE={mse[i]:.4f}, MAE={mae[i]:.4f}")

print(f"Trung bình MSE: {np.mean(mse):.4f}")
print(f"Trung bình MAE: {np.mean(mae):.4f}")


X shape: (3123, 7)
y shape: (3123, 5)
Ngày 1: MSE=2.9767, MAE=1.2704
Ngày 2: MSE=6.2263, MAE=1.9013
Ngày 3: MSE=7.5937, MAE=2.1234
Ngày 4: MSE=8.4063, MAE=2.2642
Ngày 5: MSE=9.1083, MAE=2.3556
Trung bình MSE: 6.8623
Trung bình MAE: 1.9830
